In [2]:
from preprocess import preprocess,view_head
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf


In [2]:
x_dim = 216
y_dim = 288
iterations = 100
deep = True
df = preprocess(x_dim,y_dim,deep,iterations)

KeyboardInterrupt: 

In [11]:
df

,Unnamed: 0,img_card_arr,card_class,suit,is_red,suit_num,card_string,card_number
0,0,[22 24 24 ... 11 10 12],King of Spades,Spades,0,3,King of Spades,50
1,1,[ 6 8 9 ... 20 20 26],4 of Spades,Spades,0,3,4 of Spades,41
2,2,[26 28 28 ... 8 7 9],Jack of Spades,Spades,0,3,Jack of Spades,48
3,3,[ 7 11 12 ... 0 0 0],4 of Clubs,Clubs,0,1,4 of Clubs,15
4,4,[63 70 67 ... 5 6 6],10 of Hearts,Hearts,1,2,10 of Hearts,34
...,...,...,...,...,...,...,...,...
2751,2751,[ 6 7 8 ... 228 230 218],5 of Spades,Spades,0,3,5 of Spades,42
2752,2752,[15 14 16 ... 3 3 3],6 of Diamonds,Diamonds,1,0,6 of Diamonds,4
2753,2753,[ 5 5 5 ... 25 29 34],3 of Spades,Spades,0,3,3 of Spades,40
2754,2754,[ 6 5 7 ... 8 7 11],Jack of Diamonds,Diamonds,1,0,Jack of Diamonds,9


In [ ]:
#BEFORE MODELLING< VIEW DISTRIBUTIONS


In [ ]:
df

In [ ]:
y = df['suit_num']

In [ ]:
reshaped = []
for card_arr_to_resize in df['img_card_arr']:
     reshaped.append( card_arr_to_resize.reshape(x_dim,y_dim,3))
reshaped = np.array(reshaped)
X = reshaped

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size = .8,stratify=y)

In [ ]:
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')

In [ ]:
# Modify the X values to be 0-1 instead of 0-255
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [ ]:
# Specific neural network models & layer types
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [ ]:
CNN_model = Sequential()
num_classes = 5 # red, not red
# and Dropout to reduce overfitting
CNN_model.add(Conv2D(256, kernel_size=(3, 3), activation = 'relu', input_shape = (216, 288, 3)))
CNN_model.add(MaxPooling2D(pool_size=(2, 2)))
CNN_model.add(Dropout(0.25))

CNN_model.add(Conv2D(64, (3, 3), activation='relu'))
CNN_model.add(MaxPooling2D(pool_size=(2, 2)))
CNN_model.add(Dropout(0.25))

# Flatten the output of our convolutional layers
CNN_model.add(Flatten())

# Add dense layers
CNN_model.add(Dense(128, activation='relu'))
CNN_model.add(Dense(64, activation='relu'))
CNN_model.add(Dense(num_classes, activation='softmax'))

# Print out a summary of the network
CNN_model.summary()

In [ ]:
CNN_model.compile(loss = 'sparse_categorical_crossentropy',
                  optimizer = 'Adam',
                  metrics = ['accuracy'])

In [ ]:

CNN_model.fit(X_train, y_train,
              batch_size = 5,
              epochs = 10,
              verbose = 1,
              validation_split = 0.2)

In [ ]:
score = CNN_model.evaluate(X_test, y_test, verbose=1)
score2 = CNN_model.evaluate(X_train, y_train, verbose=1)

print('Test loss:', score[0],score2[0])
print('Test accuracy:', score[1],score2[1])

In [ ]:
y1

In [ ]:
X1

In [ ]:

reshaped = []
for card_arr_to_resize in X1:
     reshaped.append( card_arr_to_resize.reshape(x_dim,y_dim,3))
reshaped = np.array(reshaped)
X1 = reshaped

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X1,y1,train_size = .76,stratify=y)

print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')

# Modify the X values to be 0-1 instead of 0-255
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
#X_train /= 255
#X_test /= 255

# Specific neural network models & layer types
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

CNN_model = Sequential()
num_classes = 5 # red, not red
# and Dropout to reduce overfitting
CNN_model.add(Conv2D(256, kernel_size=(3, 3), activation = 'relu', input_shape = (288, 288, 3)))
CNN_model.add(MaxPooling2D(pool_size=(2, 2)))
CNN_model.add(Dropout(0.25))

CNN_model.add(Conv2D(64, (3, 3), activation='relu'))
CNN_model.add(MaxPooling2D(pool_size=(2, 2)))
CNN_model.add(Dropout(0.25))

# Flatten the output of our convolutional layers
CNN_model.add(Flatten())

# Add dense layers
CNN_model.add(Dense(128, activation='relu'))
CNN_model.add(Dense(64, activation='relu'))
CNN_model.add(Dense(num_classes, activation='softmax'))

# Print out a summary of the network
CNN_model.summary()





# Compile the model with the desired loss function, optimizer, and metric(s) to track
CNN_model.compile(loss = 'sparse_categorical_crossentropy',
                  optimizer = 'Adam',
                  metrics = ['accuracy'])

# Fit the model on the training data, defining desired batch_size & number of epochs,
# running validation after each batch
# THIS WILL TAKE A LONG TIME TO RUN!!!
CNN_model.fit(X_train, y_train,
              batch_size = 5,
              epochs = 10,
              verbose = 1,
              validation_split = 0.2)

# Evaluate the model's performance on the test data
score = CNN_model.evaluate(X_test, y_test, verbose=1)
score2 = CNN_model.evaluate(X_train, y_train, verbose=1)

print('Test loss:', score[0],score2[0])
print('Test accuracy:', score[1],score2[1])








In [ ]:
plt.imshow((X_test[0]).astype(np.uint8))
CNN_model.predict_classes(X_test[0].reshape(1,288,288,3),batch_size=1)

# PLOT ACC AND LOSS ACCFOS ITERATIONS